# Data Ingest

This section reads in the training, weather, holiday and metadata for all Sites across datasets

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import numpy as np
import logging
from tqdm import tqdm
from sqlalchemy import create_engine
from utilities import convert_to_date
from utilities import get_year, get_month, string_to_timestamp

In [2]:
holidays = pd.read_csv('data/power-laws-detecting-anomalies-in-usage-holidays.csv', sep=';')
weather = pd.read_csv('data/power-laws-detecting-anomalies-in-usage-weather.csv', sep=';')
consumption = pd.read_csv('data/power-laws-detecting-anomalies-in-usage-training-data.csv', sep=';')
metadata = pd.read_csv('data/power-laws-detecting-anomalies-in-usage-metadata.csv', sep=';')

Data filtered for Site 38

In [3]:
holidays_38 = holidays[holidays['site_id'] == '038']
weather_38 = weather[weather['site_id'] == 38]
meters = ['38_0', '38_1', '38_10106', '38_10107', '38_10108', '38_10109', '38_10110', '38_10111', '38_10112']
consumption_38 = consumption[consumption['meter_id'].isin(meters)]
metadata_38 = metadata[metadata['meter_id'].isin(meters)]

# Data Wrangle
This section merges the holidays, weather, consumption and metadata for Site 38

In [4]:
holidays_38.groupby('site_id').count()

,Date,Holiday
site_id,,
038,99,99


In [5]:
weather_38.groupby('site_id').count()
weather_38['Date'] = weather_38['Timestamp'].apply(convert_to_date)
weather_38.head(5)

,Timestamp,Temperature,Distance,site_id,Date
0,2012-06-21T04:00:00-04:00,19.9,11.902932,38.0,2012-06-21
1,2012-06-23T15:00:00-04:00,26.6,11.902932,38.0,2012-06-23
2,2012-06-24T05:00:00-04:00,19.4,11.902932,38.0,2012-06-24
3,2012-06-24T13:00:00-04:00,28.7,11.902932,38.0,2012-06-24
4,2012-06-25T15:00:00-04:00,25.7,11.902932,38.0,2012-06-25


In [6]:
consumption_38.groupby('meter_id').count()

,Timestamp,Values
meter_id,,
38_0,254641,10878
38_1,254641,42240
38_10106,254641,248984
38_10107,254641,223517
38_10108,254641,249674
38_10109,254641,249672
38_10110,254641,249561
38_10111,254641,246761
38_10112,254641,247263


In [7]:
metadata_38.groupby('meter_id').count()
metadata_38

,site_id,meter_id,meter_description,units,surface,activity
24,038,38_10107,laboratory,kWh,NaN,laboratory
25,038,38_10110,other,kWh,NaN,laboratory
26,038,38_10111,cold group,kWh,NaN,laboratory
38,038,38_10106,guardhouse,kWh,NaN,laboratory
39,038,38_10112,compressed air,kWh,NaN,laboratory
61,038,38_10108,laboratory,kWh,NaN,laboratory
142,038,38_10109,lighting,kWh,NaN,laboratory
152,038,38_0,outside temperature,degree celsius,NaN,general
153,038,38_1,total workers,count,NaN,general


In [33]:
weather_holiday = pd.merge(holidays_38, weather_38,
                           on='Date', how='outer')
weather_holiday = weather_holiday.drop_duplicates(subset='Timestamp', keep="last")
(weather_holiday.groupby('Timestamp', as_index=False)['Date']
   .agg({'count': 'count'})
   .query('count > 1'))
years = ['2012', '2013', '2014', '2015', '2016', '2017']
weather_holiday['Year'] = weather_holiday['Date'].apply(get_year)
weather_holiday['Month'] = weather_holiday['Date'].apply(get_month)
weather_holiday = weather_holiday[weather_holiday['Year'].isin(years)]
weather_holiday.drop('site_id_y', inplace=True, axis=1)
weather_holiday.groupby('Year').count()

,Date,Holiday,site_id_x,Timestamp,Temperature,Distance,Month
Year,,,,,,,
2012,8535,264,264,8535,8535,8535,8535
2013,8646,263,263,8646,8646,8646,8646
2014,8420,247,247,8420,8420,8420,8420
2015,8572,246,246,8572,8572,8572,8572
2016,8710,245,245,8710,8710,8710,8710
2017,8271,218,218,8271,8271,8271,8271


In [21]:
consumption_metadata = pd.merge(consumption_38, metadata_38,
                                on='meter_id', how='outer')

In [ ]:
weather_holiday_final = pd.DataFrame()
for timestamp in weather_holiday['Timestamp'].tolist():
    weather_holiday_interim = weather_holiday[weather_holiday['Timestamp'] == timestamp]
    weather_holiday_interim['Timestamp'] = weather_holiday_interim['Timestamp'].apply(string_to_timestamp)
    weather_holiday_interim = weather_holiday_interim.reindex(np.repeat(weather_holiday_interim.index.values, 4), method='ffill')
    weather_holiday_interim['Timestamp'] += pd.TimedeltaIndex([0,15,30,45], unit='m')
    weather_holiday_final = weather_holiday_final.append(weather_holiday_interim, ignore_index=True)
weather_holiday_final.to_csv("data/power-laws-weather-holiday.csv", index=False)
weather_holiday_final.head(100)

In [28]:
weather_holiday_dataset = pd.read_csv('data/power-laws-weather-holiday.csv')
weather_holiday_dataset.groupby('Year').count()

,Date,Holiday,site_id_x,Timestamp,Temperature,Distance,site_id_y,Month
Year,,,,,,,,
2012,34140,1056,1056,34140,34140,34140,34140,34140


In [25]:
consumption_metadata['Date'] = consumption_metadata['Timestamp'].apply(convert_to_date)
consumption_metadata['Year'] = consumption_metadata['Date'].apply(get_year)

In [26]:
consumption_metadata['Timestamp'] = consumption_metadata['Timestamp'].apply(string_to_timestamp)
consumption_metadata = consumption_metadata[consumption_metadata['Year'].isin(years)]
consumption_metadata.head(5)

,meter_id,Timestamp,Values,site_id,meter_description,units,surface,activity,Date,Year
15,38_10107,2012-01-01 01:30:00,-74622.0,038,laboratory,kWh,NaN,laboratory,2012-01-01 01:30:00,2012
16,38_10107,2012-01-01 17:30:00,-74622.0,038,laboratory,kWh,NaN,laboratory,2012-01-01 17:30:00,2012
17,38_10107,2012-01-01 19:00:00,-74622.0,038,laboratory,kWh,NaN,laboratory,2012-01-01 19:00:00,2012
18,38_10107,2012-01-01 21:45:00,-74622.0,038,laboratory,kWh,NaN,laboratory,2012-01-01 21:45:00,2012
19,38_10107,2012-01-01 23:15:00,-74622.0,038,laboratory,kWh,NaN,laboratory,2012-01-01 23:15:00,2012


### Merge Consumption, Metadata, Weather, Holiday

In [ ]:
consumption_weather = pd.merge(consumption_metadata, weather_holiday,
                               on='Timestamp', how='outer')
# consumption_weather = consumption_weather[consumption_weather['Year_x'] == '2014']
columns = ['site_id_x', 'Year_x', 'Date_y','Month', 'Year_y']
consumption_weather.drop(columns, inplace=True, axis=1)
consumption_weather.head(5)